<a href="https://colab.research.google.com/github/hobezhang/NLP-based-information-retrieval-system/blob/main/Splade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = ['The sun sets over the horizon, painting the sky in warm hues.',
          'In a bustling city, people hurry to their workplaces in the early morning.',
          'A melody from a distant guitar fills the air on a quiet evening.',
          'Exploring the pages of a captivating novel, lost in a world of imagination.',
          'Raindrops tap gently on the window pane, creating a soothing rhythm.'
         ]

# 定义用于预处理文本的函数
def preprocess_text(text):
    # 移除标点符号和其他非字母数字字符
    text =  re.sub('[^a-zA-Z]', ' ', text)
    # 将文本标记化为单词
    words = word_tokenize(text.lower())
    # 移除停用词
    words = [word for word in words if word not in stopwords.words('english')]
    # print(stopwords.words('english'))
    # 将单词重新连接成字符串
    return ' '.join(words)

# 预处理样例文档
corpus = [preprocess_text(doc) for doc in corpus]
print('文档集: \n{}'.format(corpus))

# 创建一个 TfidfVectorizer 对象并将其拟合到预处理的文档集
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

# 获取与 TF-IDF 矩阵中的列对应的特征名称列表
print("特征名称:\n", vectorizer.get_feature_names_out())

# 将预处理的文档集转换为 TF-IDF 矩阵
tf_idf_matrix = vectorizer.transform(corpus)

# 打印生成的矩阵
print()
print(tf_idf_matrix)
print("TF-IDF 矩阵:\n",tf_idf_matrix.toarray())


文档集: 
['sun sets horizon painting sky warm hues', 'bustling city people hurry workplaces early morning', 'melody distant guitar fills air quiet evening', 'exploring pages captivating novel lost world imagination', 'raindrops tap gently window pane creating soothing rhythm']
特征名称:
 ['air' 'bustling' 'captivating' 'city' 'creating' 'distant' 'early'
 'evening' 'exploring' 'fills' 'gently' 'guitar' 'horizon' 'hues' 'hurry'
 'imagination' 'lost' 'melody' 'morning' 'novel' 'pages' 'painting' 'pane'
 'people' 'quiet' 'raindrops' 'rhythm' 'sets' 'sky' 'soothing' 'sun' 'tap'
 'warm' 'window' 'workplaces' 'world']

  (0, 32)	0.3779644730092272
  (0, 30)	0.3779644730092272
  (0, 28)	0.3779644730092272
  (0, 27)	0.3779644730092272
  (0, 21)	0.3779644730092272
  (0, 13)	0.3779644730092272
  (0, 12)	0.3779644730092272
  (1, 34)	0.3779644730092272
  (1, 23)	0.3779644730092272
  (1, 18)	0.3779644730092272
  (1, 14)	0.3779644730092272
  (1, 6)	0.3779644730092272
  (1, 3)	0.3779644730092272
  (1, 1)	0.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
import math

class BM25:
    def __init__(self, corpus, k1=1.5, b=0.75):
        """

        参数：
        - corpus: 文档集合，每个文档应该是一个单词列表
        - k1: 一个调节参数，控制词项频率对得分的影响
        - b: 一个调节参数，控制文档长度对得分的影响
        """
        self.corpus = corpus
        self.k1 = k1
        self.b = b
        self.doc_lengths = [len(doc) for doc in corpus]
        self.avg_doc_length = sum(self.doc_lengths) / len(corpus)
        self.doc_count = len(corpus)
        self.idf = self.calculate_idf()

    def calculate_idf(self):
        """
        计算每个词项的逆文档频率（IDF）

        返回一个字典，其中键是词项，值是对应的IDF值
        """
        idf = {}
        for doc in self.corpus:
            unique_terms = set(doc)
            for term in unique_terms:
                idf[term] = idf.get(term, 0) + 1

        for term in idf:
            idf[term] = math.log((self.doc_count - idf[term] + 0.5) / (idf[term] + 0.5) + 1.0)

        return idf

    def calculate_score(self, query, doc):
        """
        计算查询与文档之间的BM25得分

        参数：
        - query: 查询词项列表
        - doc: 文档词项列表

        返回BM25得分
        """
        score = 0.0

        for term in query:
            if term in doc:
                df = sum([1 for d in self.corpus if term in d])
                idf_term = self.idf.get(term, 0)
                score += (idf_term * doc.count(term) * (self.k1 + 1)) / (
                        doc.count(term) + self.k1 * (1 - self.b + self.b * len(doc) / self.avg_doc_length))
                score *= df / (df + 1)  # length normalization

        return score

    def rank_documents(self, query):
        """
        对文档进行排名，返回排名后的文档列表和对应的BM25得分

        参数：
        - query: 查询词项列表

        返回排名后的文档索引列表和对应的BM25得分
        """
        scores = [(i, self.calculate_score(query, self.corpus[i])) for i in range(self.doc_count)]
        ranked_docs = sorted(scores, key=lambda x: x[1], reverse=True)
        return [(doc[0], doc[1]) for doc in ranked_docs]


# 示例用法
corpus = [
    ["apple", "banana", "orange", "apple"],
    ["banana", "orange", "orange","apple"],
    ["apple", "apple", "banana"]
]

query = ["apple", "orange"]

bm25_model = BM25(corpus)
result = bm25_model.rank_documents(query)
print("排名后的文档索引和BM25得分：", result)


排名后的文档索引和BM25得分： [(1, 0.499055695673375), (0, 0.39369287621150584), (2, 0.15194951574514637)]


SPLADE

In [10]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

model_id = 'naver/splade-cocondenser-ensembledistil'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

In [ ]:
tokens = tokenizer("The dogs like fishes.", return_tensors='pt')
output = model(**tokens)
output.logits.shape

torch.Size([1, 7, 30522])

In [ ]:
import torch

vec = torch.max(
    torch.log(
        1 + torch.relu(output.logits)
    ) * tokens.attention_mask.unsqueeze(-1),
dim=1).values.squeeze()



In [ ]:
vec

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SqueezeBackward0>)

In [ ]:

cols = vec.nonzero().squeeze().cpu().tolist()
print(len(cols))


weights = vec[cols].cpu().tolist()

sparse_dict = dict(zip(cols, weights))
sparse_dict

47


{1024: 0.3254936933517456,
 1996: 0.13122430443763733,
 2027: 0.5024430155754089,
 2066: 1.8275706768035889,
 2123: 0.0026543643325567245,
 2215: 0.6085304021835327,
 2298: 0.0978935956954956,
 2300: 0.05859898775815964,
 2427: 0.12191396951675415,
 2748: 0.058032430708408356,
 3869: 2.25282621383667,
 3899: 2.5785505771636963,
 4111: 0.5009175539016724,
 4153: 0.28826817870140076,
 4176: 0.2985142469406128,
 4521: 0.7205590605735779,
 4669: 1.048478364944458,
 4743: 0.1843896210193634,
 5074: 0.031307652592659,
 5248: 0.6992028951644897,
 5437: 0.23216332495212555,
 5438: 0.1383255124092102,
 5532: 0.4857339560985565,
 5645: 1.7455496788024902,
 5742: 0.6805999875068665,
 5875: 0.2832365036010742,
 5933: 0.3116060197353363,
 6077: 2.5794029235839844,
 6240: 0.009563819505274296,
 6805: 0.25153931975364685,
 6816: 0.1546255648136139,
 7488: 0.07079166173934937,
 7529: 0.0016832482069730759,
 8843: 0.31764745712280273,
 9004: 0.28628262877464294,
 9201: 0.19111168384552002,
 9544: 0.517

In [ ]:
# extract the ID position to text token mappings
idx2token = {
    idx: token for token, idx in tokenizer.get_vocab().items()
}
# map token IDs to human-readable tokens
sparse_dict_tokens = {
    idx2token[idx]: round(weight, 2) for idx, weight in zip(cols, weights)
}
# sort so we can see most relevant tokens first
sparse_dict_tokens = {
    k: v for k, v in sorted(
        sparse_dict_tokens.items(),
        key=lambda item: item[1],
        reverse=True
    )
}
sparse_dict_tokens

#Caravanserais were roadside inns that were built along the Silk Road in areas including China, North Africa and the Middle East.

{'dog': 2.58,
 'dogs': 2.58,
 'fishes': 2.34,
 'fish': 2.25,
 'like': 1.83,
 'fishing': 1.75,
 'liked': 1.05,
 'eat': 0.72,
 'behavior': 0.7,
 'swimming': 0.68,
 'want': 0.61,
 'prefer': 0.52,
 'they': 0.5,
 'animal': 0.5,
 'desert': 0.49,
 'shark': 0.34,
 ':': 0.33,
 'breed': 0.32,
 'hunting': 0.31,
 'animals': 0.3,
 'ocean': 0.29,
 'pet': 0.29,
 'interesting': 0.28,
 'bite': 0.25,
 'smell': 0.23,
 'zoo': 0.19,
 'attract': 0.19,
 'catfish': 0.19,
 'bird': 0.18,
 'predator': 0.18,
 'tiger': 0.15,
 'feed': 0.14,
 'the': 0.13,
 'species': 0.12,
 'look': 0.1,
 'swim': 0.09,
 'bully': 0.08,
 'parasite': 0.08,
 'snake': 0.07,
 'water': 0.06,
 'yes': 0.06,
 'roger': 0.03,
 'aggression': 0.03,
 'meat': 0.01,
 'dolphin': 0.01,
 'don': 0.0,
 '##fish': 0.0}

In [ ]:
texts = [
    "Caravanserais were also an important marketplace for commodities",
    "From the 10th century onwards,as merchant and travel routes become more developed",
    "Photosynthesis is the process of storing light energy as chemical energy in cells",
]

In [ ]:
tokens = tokenizer(
    texts, return_tensors='pt',
    padding=True, truncation=True
)
output = model(**tokens)
# aggregate the token-level vecs and transform to sparse
vecs = torch.max(
    torch.log(1 + torch.relu(output.logits)) * tokens.attention_mask.unsqueeze(-1), dim=1
).values.squeeze().detach().cpu().numpy()
vecs.shape

NameError: name 'tokenizer' is not defined

In [ ]:
import numpy as np

sim = np.zeros((vecs.shape[0], vecs.shape[0]))

for i, vec in enumerate(vecs):
    sim[i,:] = np.dot(vec, vecs.T) / (
        np.linalg.norm(vec) * np.linalg.norm(vecs, axis=1)
    )
sim.round(decimals=2)

NameError: name 'vecs' is not defined